In [1]:
import brightway2 as bw
import numpy as np
from copy import copy, deepcopy

# Notes on Agribalyse 1.3

- Not all production activities in the technosphere are equal (or close) to 1.  This does not affect values of LCIA scores,  but is inconsistent with usual practices. Rescaling of these activities would require rescaling of all the corresponding exchanges, as well as their uncertainty information. This part is not implemented here, see code under 3.vii). If production activities are not 1, and one of the databases is linked to agribalyse, it might cause problems in monte carlo simulations.

- In the lognormally distributed exchanges, `loc` is equal to `amount`, which is incorrect.  At the moment `loc` is set to a logarithm of the `amount`,  which means that `amount` is equal to the median of the distribution.  It can also be chosen as a distribution mean, in which case `loc = logarithm(amount) - scale2/2`.  The choice has been made to be consistent with ecoinvent, see code under 3.v)

- Many activities in Agribalyse 1.3 have repeating exchanges.  Ideally, input activities in each one of these exchanges should be modeled as separate activities with a name, specifying what exactly the activity models. This issue has not been addresed here, so it is unclear which one of the repeating exchanges is used in the computations of LCA scores.

# Setup and TODOs

### TODO -> choose project

In [2]:
project = 'Agribalyse - ecoinvent 3.7.1 import'
bw.projects.set_current(project)

In [3]:
bw.databases

Databases dictionary with 0 objects

### TODO -> give paths to ecoinvent and agribalyse files

In [ ]:
# works for ecoinvent 3.6 and 3.7.1. maybe also for other versions, but only 36 and 371 have been tested
ei_name = "ecoinvent 3.7.1 cutoff" 
ei_path = 'link/to/ecoinvent 3.7.1 cutoff/datasets'
ag13_path = 'Agribalyse CSV FINAL_no links_Nov2016v3.CSV' # need to apply for this file

# Import databases

## 1. Biosphere3

In [5]:
%%time
bw.bw2setup()

Creating default biosphere



Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.01 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/04/2021 17:18:35
  Finished: 03/04/2021 17:18:35
  Total time elapsed: 00:00:00
  CPU %: 101.30
  Memory %: 0.92
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 1.13 seconds
Wrote 850 LCIA methods with 219059 characterization factors
Creating core data migrations

CPU times: user 1min 9s, sys: 3.7 s, total: 1min 12s
Wall time: 1min 13s


## 2. Ecoinvent

In [6]:
%%time

if ei_name in bw.databases:
    print(ei_name + " datasbase already present!!! No import is needed")    
else:
    ei = bw.SingleOutputEcospold2Importer(ei_path, ei_name)
    ei.apply_strategies()
    ei.match_database(db_name='biosphere3',fields=('name', 'category', 'unit', 'location'))
    ei.statistics()
    if len(list(ei.unlinked))==0:
        ei.write_database()
    else:
        print('Unlinked exchanges!')
        
bw.databases

Extracting XML data from 19128 datasets
Extracted 19128 datasets in 30.39 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: 

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:44


Title: Writing activities to SQLite3 database:
  Started: 03/04/2021 17:20:27
  Finished: 03/04/2021 17:21:12
  Total time elapsed: 00:00:44
  CPU %: 94.80
  Memory %: 9.78
Created database: ecoinvent 3.7.1 cutoff
CPU times: user 2min 5s, sys: 11.6 s, total: 2min 17s
Wall time: 2min 35s


Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent 3.7.1 cutoff

## 3. Agribalyse 1.3


In [7]:
# Give the name of the ecoinvent database to which Agribalyse should be linked
# works smoothly for ecoinvent 3.6 cutoff, code might need modifications for other versions
ag13_name = 'Agribalyse 1.3'
ag13_ei_name = ag13_name + ' - ' + ei_name
ag13_ei_name

'Agribalyse 1.3 - ecoinvent 3.7.1 cutoff'

In [ ]:
# try:
#     del bw.databases[ag13_ei_name]
# except:
#     pass

### i) Following steps are similar to [Aribalyse 1.2 import](https://nbviewer.jupyter.org/urls/bitbucket.org/cmutel/brightway2/raw/default/notebooks/IO%20-%20Importing%20Agribalyse%201.3%20with%20Ecoinvent%203.2%20cutoff.ipynb)

In [8]:
ag13_ei = bw.SimaProCSVImporter(ag13_path, ag13_ei_name)
ag13_ei.apply_strategies()
# Apply all migrations with previous versions of ecoinvent
ag13_ei.migrate('simapro-ecoinvent-3.3')
# Update US locations
from bw2io.strategies.locations import update_ecoinvent_locations
ag13_ei = update_ecoinvent_locations(ag13_ei)
# Biosphere flows
ag13_ei_new_biosphere_name = ag13_ei_name + " - new biosphere"
if ag13_ei_new_biosphere_name in bw.databases:
    del bw.databases[ag13_ei_new_biosphere_name]
bw.Database(ag13_ei_new_biosphere_name).register()
ag13_ei.add_unlinked_flows_to_biosphere_database(ag13_ei_new_biosphere_name)
# Add unlinked waste flows as new activities
ag13_ei.add_unlinked_activities()
ag13_ei.match_database(ei_name, fields=('reference product','location', 'unit', 'name'))
ag13_ei.statistics()

Extracted 1189 unallocated datasets in 3.64 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applying strategy: fix_zero_allocation_products
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: set_code_by_activity_hash
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: change_electricity_unit_mj_to_kwh
Applying strategy: set_lognormal_loc_value_uncertainty_safe
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: migrate_exchanges
Applying strategy: fix_localized_water_flows
A

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/04/2021 17:22:35
  Finished: 03/04/2021 17:22:35
  Total time elapsed: 00:00:00
  CPU %: 106.50
  Memory %: 11.32
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
1700 datasets
115259 exchanges
887 unlinked exchanges
  Type technosphere: 80 unique unlinked exchanges


(1700, 115259, 887)

### ii) Define some of the migrations manually.
### TODO -> check if migrations are correct

- These migrations are specific to agribalyse 1.3 + ecoinvent 3.6 combination
- Most of them are minor changes in names of activities and reference products
- Some activities contain `multiplier` field if the unit conversion is needed or reference products are not identical

In [9]:
agribalyse13_ecoinvent_names = {
    'fields': ['name'],
    'data': [
        (
            ['extrusion production, plastic film'],
            {
                'name': 'extrusion, plastic film'
            }
        ),
        (
            ['extrusion production, plastic pipes'],
            {
                'name': 'extrusion, plastic pipes'
            }
        ),
        (
            ['blow moulding production'],
            {
                'name': 'blow moulding'
            }
        ),
        (
            ['calendering production, rigid sheets'],
            {
                'name': 'calendering, rigid sheets'
            }
        ),
        (
            ['steam production in chemical industry'],
            {
                'name': 'steam production, in chemical industry',
                'reference product': 'steam, in chemical industry',
                'unit': 'kilogram',
                'multiplier': 1/2.75, # change of units, see comment on this activity in ecoinvent
                # TODO does this multiplier also change uncertainty distribution??
            }
        ),
        (
            ['water production, completely softened, from decarbonised water, at user'],
            {
                'name': 'water production, completely softened',
                'reference product': 'water, completely softened'
            }
        ),
        (
            ['market for water, decarbonised, at user'],
            {
                'name': 'market for water, decarbonised',
                'reference product': 'water, decarbonised'
            }
        ),
        (
            ['textile production, jute'],
            {
                'name': 'textile production, jute, weaving'
            }
        ),
        (
            ['market for transport, freight, sea, transoceanic ship'],
            {
                'name': 'market for transport, freight, sea, container ship with reefer, cooling',
                'reference product': 'transport, freight, sea, container ship with reefer, cooling'
            }
        ),
        (
            ['transport, freight, sea, transoceanic ship'],
            {
                'name': 'transport, freight, sea, container ship with reefer, cooling',
                'reference product': 'transport, freight, sea, container ship with reefer, cooling'
            }
        ),
        (
            ['oxidation of manganese dioxide'],
            {
                'reference product': 'potassium permanganate',
                'multiplier': 90/1000 # change of reference product, see comment on this activity in ecoinvent
            }
        ),
        (
            ['irrigation'],
            {
                'name': 'market for irrigation',
            }
        ),
    ]
}

bw.Migration("agribalyse13-ecoinvent-names").write(
    agribalyse13_ecoinvent_names,
    description="Change names of some activities"
)
ag13_ei.migrate('agribalyse13-ecoinvent-names')
ag13_ei.match_database(ei_name, fields=('reference product','location', 'unit', 'name'))

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: link_iterable_by_fields


### iii) Define migrations for market groups

In [10]:
agribalyse13_ecoinvent_market_groups = {
    'fields': ['name', 'location'],
    'data': [
        (
            ['market for electricity, low voltage', 'BR'],
            {
                'name': 'market group for electricity, low voltage'
            }
        ),
        (
            ['market for electricity, medium voltage', 'BR'],
            {
                'name': 'market group for electricity, medium voltage'
            }
        ),
    ]
}

bw.Migration("agribalyse13-ecoinvent-market-groups").write(
    agribalyse13_ecoinvent_market_groups,
    description="Change markets to market groups for Brazil"
)
ag13_ei.migrate('agribalyse13-ecoinvent-market-groups')
ag13_ei.match_database(ei_name, fields=('reference product','location', 'unit', 'name'))

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: link_iterable_by_fields


In [11]:
ag13_ei.statistics()

1700 datasets
115259 exchanges
693 unlinked exchanges
  Type technosphere: 66 unique unlinked exchanges


(1700, 115259, 693)

### iv) Allocate by production volume

In [12]:
from utils import *

In [13]:
def create_location_mapping(ag13_ei, ei_name):
    """
    Create mapping between unlinked exchanges and ecoinvent activities that constitute each unlinked exchange.
    In this case we don't need to do it manually since all unlinked exchanges need to be allocated geographically.
    Example: (market for lime, GLO) is split by production volume into 
             (market for lime, RoW) & (market for lime, RER).
    Mapping is a list of dictionaries where each dictionary corresponds to an unlinked exchange.
    The key is the name of the unlinked exchange and the values are ecoinvent activities codes.
    """
    ei = bw.Database(ei_name)
    unlinked_list = list(ag13_ei.unlinked)
    len_unlinked  = len(unlinked_list)
    mapping = [0]*len_unlinked
    for u in range(len_unlinked):
        new_el = {}
        name = unlinked_list[u]['name']
        loc  = unlinked_list[u]['location']
        acts_codes = [act['code'] for act in ei if name == act['name']]
        new_el[ (name, loc) ] = acts_codes
        mapping[u] = new_el
    return mapping

In [14]:
%%time
mapping = create_location_mapping(ag13_ei, ei_name)

CPU times: user 1min 20s, sys: 19.8 s, total: 1min 40s
Wall time: 1min 40s


In [15]:
%%time
agg = modify_exchanges(ag13_ei, mapping, ei_name)

Applying strategy: link_iterable_by_fields
CPU times: user 6.68 s, sys: 373 ms, total: 7.06 s
Wall time: 7.09 s


In [16]:
agg.statistics()

1700 datasets
117180 exchanges
0 unlinked exchanges
  


(1700, 117180, 0)

### v) Change incorrect lognormal uncertainty information
So far, all exchanges with lognormal distributions have `loc` parameter equal to `amount`, instead of `log(amount)` as required by `stats_arrays`. <br>
TODO -> Ideally need to check that all distributions make sense

In [17]:
import stats_arrays as sa

changed = []
for i,act in enumerate(agg.data):
    excs = act.get('exchanges', [])
    for j,exc in enumerate(excs):
        if exc.get('uncertainty type', False) == sa.LognormalUncertainty.id and \
        np.allclose(exc.get('amount'), exc.get('loc')):
            # a. loc chosen such that amount is the mean of the specified distribution
#             exc.update(loc=np.log(exc['amount'])-(exc['scale']**2)/2)
            # b. loc chosen such that amount is the median of the specified distribution, same as majority in ecoinvent
            exc.update(loc=np.log(exc['amount']))
            changed.append([i,j])
if "3.6" in ei_name:
    assert len(changed)==319
elif "3.7" in ei_name:
    assert len(changed)==1168

### vi) Make sure scale of lognormal is nonzero 

In [18]:
changed = []
for i,act in enumerate(agg.data):
    excs = act.get('exchanges', [])
    for j,exc in enumerate(excs):
        if exc.get('uncertainty type', False) == sa.LognormalUncertainty.id and \
        exc.get('scale')==0:
            exc.update({"uncertainty type": 0, "loc": np.nan, "scale": np.nan})
            changed.append([i,j])
assert len(changed)==6

### vii) Scale technosphere such that production exchanges are all 1

In [ ]:
# # Commented out because this code only scales production exchanges, 
# # but uncertainties in the exchanges should be scaled as well!
# acts_to_scale = []
# for act in agg.data:
#     excs = act.get('exchanges', [])
#     for exc in excs:
#         if exc.get('type') == 'production' and exc.get('amount')!=1:
#             acts_to_scale.append((act,exc.get('amount')))

# for act,production_amt in acts_to_scale:
#     excs = act.get('exchanges', [])
#     for exc in excs:
#         if exc.get('type') == 'production':
#             exc.update(amount=1)
#         else:
#             current_amt = exc.get('amount')
#             exc.update(amount=current_amt/production_amt)

### viii) Write database
It seems that now we can write the database since there are 0 unlinked exchanges, but if we actually try then we'll get an error

In [19]:
if ag13_ei_name in bw.databases:
    print(ag13_ei_name + " database already present!!! No import is needed")
else:
    agg.write_database()

NonuniqueCode: The following activities have non-unique codes: ['ammonium nitrate phosphate production', 'diammonium phosphate production', 'monoammonium phosphate production']

#### Fix non-unique codes

In [20]:
a = 'ammonium nitrate phosphate production'
b = 'diammonium phosphate production'
c = 'monoammonium phosphate production'

In [21]:
for act in agg.data:
    if act['name'] in [a,b,c]:
        print(act)

{'categories': ('Materials/fuels',), 'name': 'ammonium nitrate phosphate production', 'unit': 'kilogram', 'comment': '(1,1,5,1,1,na); assimilation MAP', 'location': 'RER', 'type': 'process', 'code': 'de221991cc69f37976042f05f448c94c', 'database': 'Agribalyse 1.3 - ecoinvent 3.7.1 cutoff'}
{'categories': ('Materials/fuels',), 'name': 'ammonium nitrate phosphate production', 'unit': 'kilogram', 'comment': '(1,1,5,1,1,na); assimilation MAP', 'location': 'RER', 'type': 'process', 'code': 'de221991cc69f37976042f05f448c94c', 'database': 'Agribalyse 1.3 - ecoinvent 3.7.1 cutoff'}
{'categories': ('Materials/fuels',), 'name': 'diammonium phosphate production', 'unit': 'kilogram', 'comment': "Mineral fertilizers. Model of transport: 'MT MAP DAP', weight transported in tons = 5,9E-02. Pedigree-Matrix='(3,3,2,1,2,na)'.", 'location': 'RER', 'type': 'process', 'code': '64037e162f6d6d1048470c3a1135f4fb', 'database': 'Agribalyse 1.3 - ecoinvent 3.7.1 cutoff'}
{'categories': ('Materials/fuels',), 'name

In [22]:
# Remove repetitive activities
agg.data.remove(
    {
        'categories': ('Materials/fuels',), 
        'name': 'ammonium nitrate phosphate production', 
        'unit': 'kilogram', 
        'comment': '(1,1,5,1,1,na); assimilation MAP', 
        'location': 'RER', 
        'type': 'process', 
        'code': 'de221991cc69f37976042f05f448c94c', 
        'database': ag13_ei_name
    }
)
agg.data.remove(
    {
        'categories': ('Materials/fuels',), 
        'name': 'diammonium phosphate production', 
        'unit': 'kilogram', 
        'comment': "Mineral fertilizers. Model of transport: 'MT MAP DAP', weight transported in tons = 5,9E-02. Pedigree-Matrix='(3,3,2,1,2,na)'.", 
        'location': 'RER', 
        'type': 'process', 
        'code': '64037e162f6d6d1048470c3a1135f4fb', 
        'database': ag13_ei_name
    }
)
agg.data.remove(
    {
        'categories': ('Materials/fuels',), 
        'name': 'monoammonium phosphate production', 
        'unit': 'kilogram', 
        'comment': '', 
        'location': 'RER', 
        'type': 'process', 
        'code': '6d61eb45c1d285073770aa839426d97c', 
        'database': ag13_ei_name
    }
)

#### Finally, write database

In [23]:
agg.statistics()

1697 datasets
117180 exchanges
0 unlinked exchanges
  


(1697, 117180, 0)

In [24]:
if ag13_ei_name in bw.databases:
    print(ag13_ei_name + " database already present!!! No import is needed")
else:
    if len(list(agg.unlinked)) == 0:
        agg.write_database()
    else:
        print("Some exchanges are still unlinked")
        print(list(agg.unlinked))

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Title: Writing activities to SQLite3 database:
  Started: 03/04/2021 17:25:28
  Finished: 03/04/2021 17:25:36
  Total time elapsed: 00:00:08
  CPU %: 96.10
  Memory %: 11.55
Created database: Agribalyse 1.3 - ecoinvent 3.7.1 cutoff
